## Unsupervised Clustering Algorithms for Empirical Population Density Based Clusters
### Pete Osorio<br>2025-06-04</br>

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.figure_factory as ff
import plotly.subplots as make_subplots
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
import rasterio
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
pio.renderers.default='notebook_connected'
pio.templates.default='simple_white'
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN, KMeans
from sklearn.metrics import silhouette_score, silhouette_samples, davies_bouldin_score
import math
from scipy.spatial.distance import cdist
import json
from geopy.distance import geodesic

In [ ]:
df = pd.read_csv('usa_pd_2020_1km.csv') #retrieved from https://hub.worldpop.org/geodata/summary?id=44771
df.rename(columns={'X': 'lon', 'Y': 'lat', 'Z': 'pop'}, inplace=True)

In [3]:
bounds_df = pd.read_json('bounds.json') # bounds_df JSON from https://observablehq.com/@rdmurphy/u-s-state-bounding-boxes
bounds_df.drop('fips', axis=1, inplace=True)
bounds_df.loc[len(bounds_df)] = ['US', [[-171.791110603, 18.91619], [-66.96466, 71.3577635769]]] # US bounding box from https://gist.github.com/graydon/11198540

# Setup the Dataframe

In [4]:
state = 'US'
lon_min = bounds_df[bounds_df['state'] == state]['bounds'].values[0][0][0]
lon_max = bounds_df[bounds_df['state'] == state]['bounds'].values[0][1][0]
lat_min = bounds_df[bounds_df['state'] == state]['bounds'].values[0][0][1]
lat_max = bounds_df[bounds_df['state'] == state]['bounds'].values[0][1][1]
df_small = df[
    (df['pop'].between(50,np.inf)) &
    (df['lat'].between(lat_min, lat_max)) &
    (df['lon'].between(lon_min, lon_max))
    ]
max_len = 100000

if df_small.shape[0] > max_len:
    df_small = df_small.sort_values(by='pop', ascending=False)[:max_len]

In [5]:
def recalculate_locales(df, bounds_df, state, n_locales, max_len, eps_value, min_samples, min_pop, dbscan_x_include = None):
    lon_min = bounds_df[bounds_df['state'] == state]['bounds'].values[0][0][0]
    lon_max = bounds_df[bounds_df['state'] == state]['bounds'].values[0][1][0]
    lat_min = bounds_df[bounds_df['state'] == state]['bounds'].values[0][0][1]
    lat_max = bounds_df[bounds_df['state'] == state]['bounds'].values[0][1][1]
    df_small = df[
        (df['pop'].between(min_pop,np.inf)) &
        (df['lat'].between(lat_min, lat_max)) &
        (df['lon'].between(lon_min, lon_max))
        ]

    if df_small.shape[0] > max_len:
        df_small = df_small.sort_values(by='pop', ascending=False)[:max_len]

    km = KMeans(n_clusters=n_locales, random_state= 970).fit(df_small[['lat', 'lon']])
    df_small['kmeans_labels'] = km.labels_
    df_small['kmeans_labels'] = df_small['kmeans_labels'].astype('category')

    gmm = GaussianMixture(n_components=n_locales, random_state= 970).fit(df_small[['lat', 'lon']].values)
    df_small['gmm_labels'] = gmm.predict(df_small[['lat', 'lon']].values)
    df_small['gmm_labels'] = df_small['gmm_labels'].astype('category')

    if dbscan_x_include == 'gmm':
        X = df_small[['lat', 'lon', 'gmm_labels']].values
    elif dbscan_x_include == 'kmeans':
        X = df_small[['lat', 'lon', 'kmeans_labels']].values
    else:
        X = df_small[['lat', 'lon', 'kmeans_labels', 'gmm_labels']].values
    X_scaled = StandardScaler().fit_transform(X)
    db = DBSCAN(eps=eps_value, min_samples=min_samples, n_jobs=-1).fit(X_scaled)
    df_small['dbscan_x_labels'] = np.where(db.labels_ != -1, db.labels_, np.nan)
    df_small['dbscan_x_labels'] = df_small['dbscan_x_labels'].astype('category')

    X = df_small[['lat', 'lon']].values
    X_scaled = StandardScaler().fit_transform(X)
    db = DBSCAN(eps=eps_value, min_samples=min_samples, n_jobs=-1).fit(X_scaled)
    df_small['dbscan_labels'] = np.where(db.labels_ != -1, db.labels_, np.nan)
    df_small['dbscan_labels'] = df_small['dbscan_labels'].astype('category')

    return df_small

In [ ]:
app = Dash()

state_options = [{"label": state, "value": state} for state in bounds_df["state"].unique()]

clusterer_options = {
    'K Means': 'kmeans_labels',
    'Gaussian Mixture': 'gmm_labels',
    'DBSCAN': 'dbscan_labels',
    'DBSCAN with Seeds': 'dbscan_x_labels'
}

seeds_options = {
    'K Means': 'kmeans',
    'Gaussian Mixture': 'gmm',
    'Both': None
}

md_text = """
# Unsupervised Clustering Algorithms for Empirical Population Density Based Clusters  
Use this applet to find the optimal geographic separation for your sales team, or sports league, or marketing campaign targets!
## Dropdown Instructions:  
* Use the state dropdownt to select a state or select the entire US.  
* Use the cluster quantity dropdown to select the number of clusters you'd like to target. This number is guaranteed for K Means and Gaussian Mixtures, however it's mearly a guideline for DBSCAN with seeds, and it has no effect on DBSCAN.  
* Use the algorithm dropdown to select your algorithm. DBSCAN with Seeds will use your seed selection to "suggest" clusters for DBSCAN to use.
* Use the seed selection dropdown to chose what to seed DBSCAN with Seeds with. You can use either K Means, Gaussian Mixture, or a combination of the two.
* Use the slider to select the minimum population (per kilomter) that you'd like to use as a base for your clusters.
* Finally, you can use the individual cluster visualizer to see how the cluster is grouped around its geographic center, where the population centers lie, and how large a radius the overal cluster has on a polar plot."""

app.layout = html.Div([
    html.Div([
        dcc.Markdown(children=md_text)
    ]),

    html.Div([
        dcc.Dropdown(
            id='state-dropdown',
            options=state_options,
            value='US',
            placeholder="Select a state",
            style={'width': '200px', 'display': 'inline-block', 'margin-right': '10px'}
        ),
        
        dcc.Dropdown(
            id='n_locales-dropdown',
            options=[{'label': str(i), 'value': i} for i in range(2, 101)],
            value=40,
            placeholder="Select the number of locales",
            style={'width': '200px', 'display': 'inline-block', 'margin-right': '10px'}
        ),

        dcc.Dropdown(
            id='clusterer-dropdown',
            options=[{"label": k, "value": v} for k, v in clusterer_options.items()],
            value='dbscan_x_labels',
            placeholder="Select a clustering algorithm",
            style={'width': '250px', 'display': 'inline-block', 'margin-right': '10px'}
        ),

        dcc.Dropdown(
            id='seeds-dropdown',
            options=[{"label": k, "value": v} for k, v in seeds_options.items()],
            value='kmeans',
            placeholder="Select a seeding algorithm",
            style={'width': '250px', 'display': 'inline-block', 'margin-right': '10px', 'color': '#1E1E1E'}
        ),

        dcc.Dropdown(
            id='cluster-dropdown',
            value=0,
            placeholder='Select a cluster to visualize',
            style={'width': '250px', 'display': 'inline-block'}
        ),

        html.Div([
            dcc.Slider(
                id='min-pop-slider',
                min=0,
                max=5000,
                step=10,
                value=2500,
                marks={i: str(i) for i in range(0, 5001, 500)},
                tooltip={"placement": "bottom", "always_visible": True}
            )
        ], style={'width': '600px', 'margin-top': '20px'})
    ], style={'display': 'flex', 'align-items': 'center', 'flex-wrap': 'wrap'}),

    html.Div([dcc.Graph(id='map-graph')]),
    html.Div([dcc.Graph(id='histogram')]),
    html.Div([dcc.Graph(id='polar-plot')]),
], style={'backgroundColor': '#1E1E1E', 'color': '#EAEAEA'})

@app.callback(
    Output('map-graph', 'figure'),
    Output('histogram', 'figure'),
    Input('state-dropdown', 'value'),
    Input('n_locales-dropdown', 'value'),
    Input('min-pop-slider', 'value'),
    Input('clusterer-dropdown', 'value'),
    Input('seeds-dropdown', 'value')
)
def update_map(selected_state, selected_n_locales, selected_min_pop, selected_clusterer, selected_seeds):
    df_out = recalculate_locales(df=df, bounds_df=bounds_df, state=selected_state, 
                                 n_locales=selected_n_locales, max_len=50000, eps_value=.25, 
                                 min_samples=30, min_pop=selected_min_pop, dbscan_x_include=selected_seeds)
    fig_map = px.scatter_map(df_out,
                         lat='lat', lon='lon', color=selected_clusterer,
                         hover_data=['pop'], title=f"The {selected_n_locales} states of {selected_state}",
                         map_style="open-street-map", opacity=0.5, height=1080*.8, width=1920*.8,
                         color_discrete_sequence=px.colors.qualitative.Dark24, zoom=4, template='plotly_dark')
    df_hist = df_out.groupby(selected_clusterer)['pop'].sum().reset_index()
    fig_hist = px.bar(df_hist, x=selected_clusterer, y='pop',
                      title=f"Population Distribution for {selected_clusterer}",
                      labels={selected_clusterer: "Cluster", 'pop': "Total Population"},
                      color=selected_clusterer,
                      color_discrete_sequence=px.colors.qualitative.Dark24,
                      width=1920*.8,
                      template='plotly_dark')

    return fig_map, fig_hist

@app.callback(
    Output('cluster-dropdown', 'options'),
    Input('clusterer-dropdown', 'value'),
    Input('state-dropdown', 'value'),
    Input('n_locales-dropdown', 'value'),
    Input('min-pop-slider', 'value'),
    Input('seeds-dropdown', 'value')
)
def update_cluster_options(selected_clusterer, selected_state, selected_n_locales, selected_min_pop, selected_seeds):
    df_out = recalculate_locales(df=df, bounds_df=bounds_df, state=selected_state, 
                                 n_locales=selected_n_locales, max_len=50000, eps_value=.25,
                                 min_samples=30, min_pop=selected_min_pop, dbscan_x_include=selected_seeds)
    clusters = df_out[selected_clusterer].unique()
    return [{"label": f"Cluster {c}", "value": c} for c in clusters]

@app.callback(
    Output('polar-plot', 'figure'),
    Input('cluster-dropdown', 'value'),
    Input('clusterer-dropdown', 'value'),
    Input('state-dropdown', 'value'),
    Input('n_locales-dropdown', 'value'),
    Input('min-pop-slider', 'value'),
    Input('seeds-dropdown', 'value')
)
def update_polar_plot(selected_cluster, selected_clusterer, selected_state, selected_n_locales, selected_min_pop, selected_seeds):
    df_out = recalculate_locales(df=df, bounds_df=bounds_df, state=selected_state, 
                                 n_locales=selected_n_locales, max_len=50000, eps_value=.25,
                                 min_samples=30, min_pop=selected_min_pop, dbscan_x_include=selected_seeds)

    cluster_df = df_out[df_out[selected_clusterer] == selected_cluster].sort_values(by='pop', ascending=False)

    centroid_lat = cluster_df['lat'].mean()
    centroid_lon = cluster_df['lon'].mean()
    centroid_coords = (centroid_lat, centroid_lon)

    cluster_df['theta'] = np.arctan2(cluster_df['lat'] - centroid_lat, cluster_df['lon'] - centroid_lon) * (180 / np.pi)
    cluster_df['r'] = cluster_df['lat'].map(lambda lat: geodesic(centroid_coords, (lat, cluster_df.loc[cluster_df['lat'] == lat, 'lon'].values[0])).miles)
    cluster_df.sort_values(by='pop', ascending=False, inplace=True)
    fig_polar = px.scatter_polar(cluster_df,
                                 r='r', theta='theta', hover_data=['pop'],
                                 title=f"Cluster {selected_cluster} Distribution", color='pop', size='pop', width=1920*.8, height=1080*.8, template='plotly_dark',
                                 color_continuous_scale=px.colors.sequential.Jet)

    return fig_polar

app.run(jupyter_model='external')

[2025-06-04 22:40:31,628] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\Pete\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Pete\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Pete\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Pete\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^